# Building Agentic AI Systems with MCP Servers & Clients from Scratch

![](https://i.imgur.com/s9AFYsn.png)

## Installing Dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.30 langgraph==0.6.5 langchain-mcp-adapters==0.1.9 mcp==1.12.4 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 989.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.7 MB/s eta 0:00:00


## Get OpenAI Key Environment File

In [ ]:
!gdown 1dyWtBg0RCg_WqzsuMWhvJwrN8_GGDmbI

Downloading...
From: https://drive.google.com/uc?id=1dyWtBg0RCg_WqzsuMWhvJwrN8_GGDmbI
To: /content/.env
100% 16.0/16.0 [00:00<00:00, 50.5kB/s]


## Add your OpenAI Key in the Environment File

Open up the `.env` file and enter your OpenAI key in the place shown below and then save the file

![](https://i.imgur.com/ThkufIW.png)

## Building a MCP Server for the Finance Department

This section creates a **Finance MCP Server** using FastMCP.  
The server runs on **port 8010** with `mcp.run(transport="streamable-http")`.

It exposes two LLM-driven finance advisory tools:

- **`ops_spend_guidance(department: str, context: str = "") -> str`**  
  Provides practical spend guidance for a department, covering approval thresholds, PO requirements, vendor payment terms, and quick guardrails.  
  Uses static snapshots from the `DEPARTMENTS` dictionary combined with GPT-4o-mini to produce tailored recommendations.

- **`recommend_budget_actions(department: str, narrative: str = "") -> str`**  
  Summarizes the department’s budget status and suggests 3–5 actions to stay on track.  
  Accepts an optional narrative to guide the analysis and outputs actionable recommendations.

The server is initialized with `FastMCP(name="FinanceServer", instructions=...)` which documents its tools for discovery by clients.


In [ ]:
%%writefile finance_mcp_server.py
from mcp.server.fastmcp import FastMCP
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Small static department snapshots used for LLM reasoning
DEPARTMENTS = {
    "Engineering": {
        "fy": "2025", "budget": 2_400_000, "spent_ytd": 980_000, "headcount": 18,
        "open_initiatives": ["Observability uplift", "Inference cost control", "Data platform hardening"]
    },
    "Operations": {
        "fy": "2025", "budget": 6_000_000, "spent_ytd": 2_850_000, "headcount": 72,
        "open_initiatives": ["Warehouse automation pilot", "Route optimization v2", "Safety training refresh"]
    },
    "Sales": {
        "fy": "2025", "budget": 3_600_000, "spent_ytd": 1_240_000, "headcount": 24,
        "open_initiatives": ["Partner channel revamp", "Pricing experiments", "New region PoC"]
    }
}

mcp = FastMCP(
    name="FinanceServer",
    instructions="""
Finance tools with LLM reasoning over small static snapshots.

TOOLS
- ops_spend_guidance(department, context=""):
  Practical guidance for Operations (or any function) on spend controls:
  approval tiers, PO usage, vendor terms, and quick guardrails.

- recommend_budget_actions(department, narrative=""):
  Short summary of budget posture and top actions to stay on plan.

All tool responses are plain text generated by the LLM.
""",
    port=8010
)

@mcp.tool()
def ops_spend_guidance(department: str, context: str = "") -> str:
    """
    Guidance on spend controls for the department (approval tiers, PO usage, vendor terms).
    Returns free-form text (bullets recommended).
    """
    snap = DEPARTMENTS.get(department)
    if not snap:
        return f"[Finance] Unknown department: {department}"
    prompt = f"""
You are a finance partner. Using the department snapshot and optional context,
give practical spend guidance: approval thresholds, PO requirements, vendor payment terms,
and any quick guardrails. Keep it short, concrete, and actionable.

Department snapshot:
{snap}

Additional context (may be empty):
{context}

Output: a short bullet list with specific recommendations and a one-line rationale where needed.
"""
    return llm.invoke(prompt).content

@mcp.tool()
def recommend_budget_actions(department: str, narrative: str = "") -> str:
    """
    Budget summary + top actions. Returns free-form text.
    """
    snap = DEPARTMENTS.get(department)
    if not snap:
        return f"[Finance] Unknown department: {department}"
    prompt = f"""
You are a finance planner. Using the department snapshot and optional narrative,
summarize the current budget posture and list 3–5 specific actions to stay on plan.
Mention amounts where useful and focus on next steps.

Department snapshot:
{snap}

Narrative (may be empty):
{narrative}

Output: comprehensive summary + numbered actions.
"""
    return llm.invoke(prompt).content

if __name__ == "__main__":
    print("Starting Finance MCP Server on http://localhost:8010/mcp ...")
    mcp.run(transport="streamable-http")

Writing finance_mcp_server.py


## Deploying the MCP Server for the Finance Department

We launch the Finance MCP server as a background process using **streamable HTTP**.  
- Endpoint: `http://localhost:8010/mcp`  
- Logs: written to `finance_server_output.log`  

Once running, its tools are accessible to the client agent for discovery and use.

In [ ]:
!nohup python /content/finance_mcp_server.py > finance_server_output.log 2>&1 &

![](https://i.imgur.com/8n4J8qV.png)

## Building a MCP Server for the HR Department

This section defines an **HR MCP Server** using FastMCP, configured to run on **port 8011** with `mcp.run(transport="streamable-http")`.

It registers two HR-focused tools:

- **`get_employee_details(employee_id: str) -> dict`**  
  Returns employee information including ID, name, role, and department.  
  Example: `{"employee_id": "123", "name": "Alice Johnson", "role": "Software Engineer", "department": "Tech"}`

- **`check_leave_balance(employee_id: str) -> dict`**  
  Provides the leave balance for a given employee.  
  Example: `{"employee_id": "123", "leave_balance": 12}`

As with Finance, the HR server includes clear instructions in its constructor for client-side discovery.

In [ ]:
%%writefile hr_mcp_server.py

from mcp.server.fastmcp import FastMCP
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Small HR snapshots
EMPLOYEES = {
    "EMP456": {
        "name": "Anita Rao", "role": "Senior Analyst", "location": "Bengaluru",
        "exempt_status": "exempt", "pto_balance_days": 12, "on_critical_project": False,
        "manager": "MGR-22"
    },
    "EMP789": {
        "name": "Vikram Shah", "role": "Ops Lead", "location": "Mumbai",
        "exempt_status": "exempt", "pto_balance_days": 5, "on_critical_project": True,
        "manager": "MGR-31"
    }
}

POLICIES = {
    "pto_accrual": "Employees accrue PTO each pay period; manager approval required. Blackout periods may apply.",
    "carryover": "Up to 10 PTO days may be carried forward to next year.",
    "critical_project_coordination": "Coordinate with project owner if employee is on a critical project.",
    "general_holidays": "Company observes standard public holidays as announced annually."
}

mcp = FastMCP(
    name="HRServer",
    instructions="""
HR tools with LLM reasoning over small static snapshots.

TOOLS
- leave_decision(employee_id, request):
  Approve / approve with conditions / deny with a brief rationale and 2–3 next steps.

- answer_policy(question):
  Short policy guidance referencing the small policy dictionary.

All responses are plain text from the LLM.
""",
    port=8011
)

@mcp.tool()
def leave_decision(employee_id: str, request: str) -> str:
    """
    request example: {"dates":"2025-09-02 to 2025-09-06","reason":"family event","backup":"Alice"}
    Returns free-form text with decision + rationale + next steps.
    """
    emp = EMPLOYEES.get(employee_id)
    if not emp:
        return f"[HR] Unknown employee_id: {employee_id}"
    prompt = f"""
You are an HR business partner. Decide on a PTO request using the employee snapshot,
request details, and policy notes. Choose approve / approve with conditions / deny,
give a short rationale, and list 2–3 next steps.

Employee snapshot:
{emp}

Leave request:
{request}

Policy notes:
{POLICIES}

Output: decision header, brief rationale, and numbered next steps.
"""
    return llm.invoke(prompt).content

@mcp.tool()
def answer_policy(question: str) -> str:
    """
    Short policy answer citing which policy keys were used.
    """
    prompt = f"""
You are an HR policy assistant. Answer briefly using the policy dictionary and cite the keys you used.

Policies:
{POLICIES}

Question:
{question}

Output: 2–4 sentences plus a final line: "Cited: <policy details>".
"""
    return llm.invoke(prompt).content

if __name__ == "__main__":
    print("Starting HR MCP Server on http://localhost:8011/mcp ...")
    mcp.run(transport="streamable-http")

Writing hr_mcp_server.py


## Deploying the MCP Server for the HR Department

The HR MCP server is started as a background process:  
- Endpoint: `http://localhost:8011/mcp`  
- Logs: stored in `hr_server_output.log`  

This brings the HR server online and ready to provide employee-related information to agents.


In [ ]:
!nohup python /content/hr_mcp_server.py > hr_server_output.log 2>&1 &

![](https://i.imgur.com/sHdQjL9.png)

## Building the Client Agent

Here we construct the **Client Agent** that will consume tools from both the Finance and HR MCP servers. Steps include:

1. Creating a `MultiServerMCPClient` that connects to the Finance (`8010`) and HR (`8011`) endpoints  
2. Discovering available tools dynamically with `get_tools()`  
3. Initializing a GPT-4o-mini model as the reasoning engine  
4. Creating a **ReAct agent** using `create_react_agent` and passing the tools and a System Prompt  

This produces a single agent capable of intelligently routing requests to the right department’s tools.

In [ ]:
%%writefile client_agent.py
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import asyncio
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

SYSTEM_PROMPT = SystemMessage(content="""
You are a helpful assistant that can call tools exposed via MCP.
- Use Finance tools for spend guidance and budget actions.
- Use HR tools for leave decisions and quick policy answers.
- Keep answers short and actionable.
- When a tool provides bullet points, preserve structure.
- If a tool returns an error string, surface it plainly.
""")

async def main():
    client = MultiServerMCPClient({
        "finance": {
            "url": "http://localhost:8010/mcp",
            "transport": "streamable_http"
        },
        "hr": {
            "url": "http://localhost:8011/mcp",
            "transport": "streamable_http"
        },
    })

    tools = await client.get_tools()
    print("Discovered tools:", [t.name for t in tools])

    agent = create_react_agent(model=llm, tools=tools, prompt=SYSTEM_PROMPT)

    # Sample prompts to try
    samples = [
        # Finance (for Operations): spend guardrails
        "We need spend guardrails for Operations. Suggest approval tiers and when to require POs.",

        # Finance: budget actions
        "Give budget actions for Engineering to stay on plan this quarter.",

        # HR: leave decision
        'HR: Decide on PTO for EMP456, with the request of {"dates":"2025-09-02 to 2025-09-06","reason":"family event","backup":"Alice"}',

        # HR: policy quick answer
        "HR policy: Can I carry forward unused PTO to next year?"
    ]

    for q in samples:
        print("\n---")
        print("User:", q)

        async for event in agent.astream(input={"messages": [HumanMessage(content=q)]},
                                         config={"recursion_limit": 25}, stream_mode="values"):
            if event:
                event["messages"][-1].pretty_print()
        print()


if __name__ == "__main__":
    asyncio.run(main())

Writing client_agent.py


## Running the Client Agent

We execute the client agent with example queries to demonstrate cross-server functionality:

- **Finance query**: “Generate an invoice for customer 123”  
- **HR query**: “Show me the leave balance for employee 456”  
- **Finance query**: “What is the tech department’s budget summary?”  

For each case, the agent selects the correct tool from the appropriate server, invokes it, and produces a natural language summary of the result.  
This confirms that one agent can seamlessly orchestrate across multiple MCP servers.


```bash
/content# python client_agent.py
Discovered tools: ['ops_spend_guidance', 'recommend_budget_actions', 'leave_decision', 'answer_policy']

---
User: We need spend guardrails for Operations. Suggest approval tiers and when to require POs.
================================ Human Message =================================

We need spend guardrails for Operations. Suggest approval tiers and when to require POs.
================================== Ai Message ==================================
Tool Calls:
  ops_spend_guidance (call_xajA51tcpqOF9Lil3DJzxWQa)
 Call ID: call_xajA51tcpqOF9Lil3DJzxWQa
  Args:
    department: Operations
================================= Tool Message =================================
Name: ops_spend_guidance

- **Approval Thresholds**:  
  - Purchases up to $5,000: Department head approval required.  
  - Purchases $5,001 - $50,000: Finance partner and department head approval required.  
  - Purchases over $50,000: Executive team approval required.  
  *Rationale: Ensures appropriate oversight for varying levels of expenditure.*

- **PO Requirements**:  
  - All purchases over $1,000 must have a Purchase Order (PO) before incurring expenses.  
  *Rationale: Maintains budget control and tracking of expenditures.*

- **Vendor Payment Terms**:  
  - Standard payment terms: Net 30 days.  
  - Early payment discounts should be negotiated where possible.  
  *Rationale: Optimizes cash flow while taking advantage of potential savings.*

- **Quick Guardrails**:  
  - Monitor spending against the budget regularly; aim to stay within 50% of the budget by mid-year.  
  - Prioritize open initiatives for funding to ensure strategic alignment.  
  *Rationale: Helps maintain financial discipline and focus on key projects.*
================================== Ai Message ==================================

Here are the spend guardrails for Operations:

- **Approval Thresholds**:  
  - Purchases up to $5,000: Department head approval required.  
  - Purchases $5,001 - $50,000: Finance partner and department head approval required.  
  - Purchases over $50,000: Executive team approval required.  

- **PO Requirements**:  
  - All purchases over $1,000 must have a Purchase Order (PO) before incurring expenses.  

- **Vendor Payment Terms**:  
  - Standard payment terms: Net 30 days.  
  - Early payment discounts should be negotiated where possible.  

- **Quick Guardrails**:  
  - Monitor spending against the budget regularly; aim to stay within 50% of the budget by mid-year.  
  - Prioritize open initiatives for funding to ensure strategic alignment.  

These guidelines will help maintain financial oversight and control.


---
User: Give budget actions for Engineering to stay on plan this quarter.
================================ Human Message =================================

Give budget actions for Engineering to stay on plan this quarter.
================================== Ai Message ==================================
Tool Calls:
  recommend_budget_actions (call_IFegXTptMXNnaj6IYPWhO1dy)
 Call ID: call_IFegXTptMXNnaj6IYPWhO1dy
  Args:
    department: Engineering

2. **Inference cost control**
3. **Data platform hardening**

These initiatives are critical for enhancing operational efficiency and controlling costs, which will be essential for staying within budget for the remainder of the fiscal year.

### Next Steps: Specific Actions to Stay on Plan

1. **Monitor Spending Against Budget**: Implement a bi-weekly review of expenditures against the budget to ensure that spending aligns with planned allocations. This will help identify any potential overspending early and allow for corrective actions.

2. **Prioritize Open Initiatives**: Assess the urgency and impact of the open initiatives. Allocate resources to the most critical projects first, particularly focusing on **Inference cost control** to manage and reduce ongoing expenses.

3. **Enhance Resource Allocation**: Evaluate the current headcount and consider whether reallocating existing personnel to high-priority initiatives could improve efficiency without increasing costs. This may involve cross-training team members to support multiple initiatives.

4. **Set Milestones for Initiatives**: Establish clear milestones and deadlines for each open initiative. This will help track progress and ensure that resources are being utilized effectively, allowing for timely adjustments if any initiative is falling behind.

5. **Engage in Cost-Benefit Analysis**: For each open initiative, conduct a cost-benefit analysis to determine the expected return on investment. This will help prioritize funding and resources towards initiatives that promise the highest impact on the department's goals.

By implementing these actions, the department can maintain its budget posture and ensure that it remains on track for the fiscal year 2025.

### Current Budget Posture Summary

- Total budget for FY 2025: **$2,400,000**
- Year-to-date (YTD) spending: **$980,000** (40.8% utilized)
- Current headcount: **18 employees**
- Open initiatives:
  1. **Observability uplift**
  2. **Inference cost control**
  3. **Data platform hardening**

### Next Steps: Specific Actions to Stay on Plan

1. **Monitor Spending Against Budget**: Implement bi-weekly reviews of expenditures to ensure alignment with planned allocations.
  
2. **Prioritize Open Initiatives**: Focus on critical projects, especially **Inference cost control**, to manage and reduce expenses.

3. **Enhance Resource Allocation**: Consider reallocating personnel to high-priority initiatives and cross-training team members.

4. **Set Milestones for Initiatives**: Establish clear milestones and deadlines for tracking progress and resource utilization.

5. **Engage in Cost-Benefit Analysis**: Conduct analyses for each initiative to prioritize funding towards those with the highest expected impact.

By following these actions, the Engineering department can stay on track with its budget for the remainder of the fiscal year.


---
User: HR: Decide on PTO for EMP456, with the request of {"dates":"2025-09-02 to 2025-09-06","reason":"family event","backup":"Alice"}
================================ Human Message =================================

HR: Decide on PTO for EMP456, with the request of {"dates":"2025-09-02 to 2025-09-06","reason":"family event","backup":"Alice"}
================================== Ai Message ==================================
Tool Calls:
  leave_decision (call_jV729UHjrA75M6R4Dvl4snRq)
 Call ID: call_jV729UHjrA75M6R4Dvl4snRq
  Args:
    employee_id: EMP456
    request: {"dates":"2025-09-02 to 2025-09-06","reason":"family event","backup":"Alice"}
================================= Tool Message =================================
Name: leave_decision

**Decision: Approve**

**Rationale:** Anita Rao has a sufficient PTO balance of 12 days and is not currently on a critical project. Her request for time off from September 2 to September 6 for a family event is reasonable and aligns with company policy.

**Next Steps:**
1. Notify Anita of the approval of her PTO request.
2. Confirm with her manager, MGR-22, that Alice will be available as a backup during her absence.
3. Update the PTO records to reflect the approved leave dates.
================================== Ai Message ==================================

**Decision: Approve**

**Rationale:** Anita Rao has a sufficient PTO balance of 12 days and is not currently on a critical project. Her request for time off from September 2 to September 6 for a family event is reasonable and aligns with company policy.

**Next Steps:**
1. Notify Anita of the approval of her PTO request.
2. Confirm with her manager, MGR-22, that Alice will be available as a backup during her absence.
3. Update the PTO records to reflect the approved leave dates.


---
User: HR policy: Can I carry forward unused PTO to next year?
================================ Human Message =================================

HR policy: Can I carry forward unused PTO to next year?
================================== Ai Message ==================================
Tool Calls:
  answer_policy (call_3VRTFiYBGV2nNNzz6SHTPoiS)
 Call ID: call_3VRTFiYBGV2nNNzz6SHTPoiS
  Args:
    question: Can I carry forward unused PTO to next year?
================================= Tool Message =================================
Name: answer_policy

Yes, you can carry forward unused PTO to the next year, but only up to 10 PTO days. It's important to keep track of your accrued days to ensure you don't exceed this limit.

Cited: {'carryover': 'Up to 10 PTO days may be carried forward to next year.'}
================================== Ai Message ==================================

Yes, you can carry forward unused PTO to the next year, but only up to 10 PTO days. It's important to keep track of your accrued days to ensure you don't exceed this limit.

/content#

```